<a href="https://colab.research.google.com/github/ValiantChiu/Computer_Vision_AICSP/blob/main/%E9%90%B5%E9%81%94%E5%B0%BC%E5%AD%98%E6%B4%BB%E7%8E%87%E7%89%B9%E5%BE%B5%E5%B7%A5%E7%A8%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 蘭婷

In [ ]:
!pip install numpy
!pip install matplotlib
!pip install pandas
!pip install seaborn
!pip install scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot') 
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
import seaborn as sns
sns.set_style('ticks')
from sklearn.ensemble import RandomForestClassifier

In [ ]:
!git clone https://github.com/agconti/kaggle-titanic.git

In [ ]:
df_train=pd.read_csv('./kaggle-titanic/data/train.csv')
df_test=pd.read_csv('./kaggle-titanic/data/test.csv')
data = df_train.append(df_test)

In [ ]:
data

特徵工程
蘭婷 

1.由於 SibSp 與 Parch 欄位所記錄的人數皆屬於家庭成員，所以我們將兩者相加創造新的特徵欄位稱家庭人數(Family_Size)。

2.觀察 Family_Size 與生存率的關係，發現當家庭人數為4時的生存率(約 72.4%)明顯高於其他人數。

In [ ]:
# 創造新的特徵變數：家庭人數(Family_Size)
data['Family_Size'] = data['SibSp'] + data['Parch'] + 1

Survival_Rate = data[['Family_Size','Survived']].groupby(by=['Family_Size']).agg(np.mean)*100
Survival_Rate.columns = ['Survival Rate(%)']
Survival_Rate.reset_index()

# 繼群

In [ ]:
from google.colab import files
files.upload()

### Load data

In [ ]:
!kaggle competitions download -c titanic
!unzip titanic.zip

In [ ]:
import pandas as pd
import seaborn as sns
import pandas

In [ ]:
df_train=pd.read_csv('/content/train.csv')
df_test=pd.read_csv('/content/test.csv')

df_train['sex_int'], _ = pd.factorize(df_train['Sex'])
df_test['sex_int'], _ = pd.factorize(df_test['Sex'])

df_train['Embarked'], _ = pd.factorize(df_train['Embarked'])
df_test['Embarked'], _ = pd.factorize(df_test['Embarked'])

### Fill Mssing Value

In [ ]:
df_train['Age'] = df_train['Age'].interpolate()
df_test['Age'] = df_test['Age'].interpolate()

In [ ]:
cabin_type = list(set([  i[0] for i in df_train.Cabin.values if type(i) != float]))
encode = range(len(cabin_type))
encode_dict = dict(zip(cabin_type, encode))
cabin_int = [i if type(i) == float else encode_dict[i[0]] for i in df_train.Cabin.values ]

df_train['cabin_int'] = cabin_int
df_train['cabin_int'] = pd.to_numeric(df_train['cabin_int'])

cabin_type = list(set([  i[0] for i in df_test.Cabin.values if type(i) != float]))
encode = range(len(cabin_type))
encode_dict = dict(zip(cabin_type, encode))
cabin_int = [i if type(i) == float else encode_dict[i[0]] for i in df_test.Cabin.values ]
df_test['cabin_int'] = cabin_int
df_test['cabin_int'] = pd.to_numeric(df_test['cabin_int'])

In [ ]:
df_train['cabin_int'] = df_train['cabin_int'].interpolate().fillna(0)
df_test['cabin_int'] = df_test['cabin_int'].interpolate().fillna(0)

### Build Model

In [ ]:
from sklearn.model_selection import train_test_split

# Select the features we want to use to predict survival
features = ['Age', 'sex_int', 'Pclass','cabin_int','Fare','SibSp','Embarked']
X = df_train[features]
y = df_train['Survived']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#### Basic model

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create a decision tree classifier
clf = DecisionTreeClassifier()

# Train the classifier on the training data
clf.fit(X_train, y_train)

In [ ]:
# Make predictions on the test data
predictions = clf.predict(X_test)

# Evaluate the model's performance
from sklearn.metrics import accuracy_score
print(f'Accuracy: {accuracy_score(y_test, predictions)}')


In [ ]:
#Final Prediction
predictions = clf.predict(df_test[features])

#### Xgboost

In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label = y_train)
dtest = xgb.DMatrix(X_test, label = y_test)
params_1 = {
    'booster': 'gbtree',
    'max_depth': 60, 
    'learning_rate': 0.2,
    'sample_type': 'uniform',
    'normalize_type': 'tree',
    'objective': 'binary:hinge',
    'rate_drop': 0.3,
    'n_estimators': 500
}
xgb_clf = xgb.train(params_1, dtrain)

predictions = xgb_clf.predict(dtest)


# Evaluate the model's performance
from sklearn.metrics import accuracy_score
print(f'Accuracy: {accuracy_score(y_test, predictions)}')

#Final Prediction
predictions = xgb_clf.predict(xgb.DMatrix(df_test[features]))

In [ ]:
predictions

#### Output result

In [ ]:
df_test['Survived'] = predictions.astype(int)

In [ ]:
df_test.loc[:,['PassengerId','Survived']].to_csv('submission.csv',index=False)

In [ ]:
!kaggle competitions submit -c titanic -f submission.csv -m "Kees Fifth prediction of AICSP program"